[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/transformers/HuggingFace%20in%20Spark%20NLP%20-%20BERT.ipynb)

## Import BERT models from HuggingFace 🤗  into Spark NLP 🚀 

Let's keep in mind a few things before we start 😊 

- This feature is only in `Spark NLP 3.1.x` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import models for BERT from HuggingFace but they have to be compatible with `TensorFlow` and they have to be in `Fill Mask` category. Meaning, you cannot use BERT models trained/fine-tuned on a specific task such as token/sequence classification.

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.4.1` version and Transformers on `4.6.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
import os

# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4
! pip install --ignore-installed -q spark-nlp==2.7.1

openjdk version "1.8.0_292"
OpenJDK Runtime Environment (build 1.8.0_292-8u292-b10-0ubuntu1~18.04-b10)
OpenJDK 64-Bit Server VM (build 25.292-b10, mixed mode)
     |████████████████████████████████| 215.7MB 63kB/s 
     |████████████████████████████████| 204kB 12.4MB/s 
     |████████████████████████████████| 143kB 5.3MB/s 


In [2]:
!pip install -q transformers==4.6.1 tensorflow==2.4.1

     |████████████████████████████████| 2.3MB 5.1MB/s 
     |████████████████████████████████| 394.3MB 29kB/s 
     |████████████████████████████████| 3.3MB 36.6MB/s 
     |████████████████████████████████| 901kB 41.6MB/s 
     |████████████████████████████████| 471kB 39.5MB/s 
     |████████████████████████████████| 2.9MB 20.0MB/s 
     |████████████████████████████████| 3.8MB 20.1MB/s 


- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [bert-base-cased](https://huggingface.co/bert-base-cased) model from HuggingFace as an example
- In addition to `TFBertModel` we also need to save the `BertTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [3]:
from transformers import TFBertModel, BertTokenizer 

MODEL_NAME = 'bert-base-uncased'

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME).save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))
model = TFBertModel.from_pretrained(MODEL_NAME).save_pretrained("./{}".format(MODEL_NAME), saved_model=True)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

INFO:tensorflow:Assets written to: ./bert-base-uncased/saved_model/1/assets


INFO:tensorflow:Assets written to: ./bert-base-uncased/saved_model/1/assets


Let's have a look inside these two directories and see what we are dealing with:

In [4]:
!ls -l {MODEL_NAME}

total 427932
-rw-r--r-- 1 root root       599 Jun 20 21:32 config.json
drwxr-xr-x 3 root root      4096 Jun 20 21:32 saved_model
-rw-r--r-- 1 root root 438190872 Jun 20 21:32 tf_model.h5


In [5]:
!ls -l {MODEL_NAME}/saved_model/1

total 7740
drwxr-xr-x 2 root root    4096 Jun 20 21:32 assets
-rw-r--r-- 1 root root 7917092 Jun 20 21:32 saved_model.pb
drwxr-xr-x 2 root root    4096 Jun 20 21:32 variables


In [6]:
!ls -l {MODEL_NAME}_tokenizer

total 236
-rw-r--r-- 1 root root    112 Jun 20 21:30 special_tokens_map.json
-rw-r--r-- 1 root root    522 Jun 20 21:30 tokenizer_config.json
-rw-r--r-- 1 root root 231508 Jun 20 21:30 vocab.txt


- as you can see, we need the SavedModel from `saved_model/1/` path
- we also be needing `vocab.txt` from the tokenizer
- all we need is to just copy the `vocab.txt` to `saved_model/1/assets` which Spark NLP will look for

In [7]:
!cp {MODEL_NAME}_tokenizer/vocab.txt {MODEL_NAME}/saved_model/1/assets

## Import and Save BERT in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [8]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2021-06-20 21:32:18--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-06-20 21:32:18--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1608 (1.6K) [text/plain]
Saving to: ‘STDOUT’

-                     0%[                    ]       0  --.-KB/s               setup Colab for PySpark 3.0.2 and Spark NLP 3.1.0
-                   100%[==

Let's start Spark with Spark NLP included via our simple `start()` function

In [9]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `BertEmbeddings` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `BertEmbeddings` in runtime, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- `setStorageRef` is very important. When you are training a task like NER or any Text Classification, we use this reference to bound the trained model to this specific embeddings so you won't load a different embeddings by mistake and see terrible results 😊
- It's up to you what you put in `setStorageRef` but it cannot be changed later on. We usually use the name of the model to be clear, but you can get creative if you want! 
- The `dimension` param is is purely cosmetic and won't change anything. It's mostly for you to know later via `.getDimension` what is the dimension of your model. So set this accordingly.


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
from sparknlp.annotator import *

bert = BertEmbeddings.loadSavedModel(
     '{}/saved_model/1'.format(MODEL_NAME),
     spark
 )\
 .setInputCols(["document",'token'])\
 .setOutputCol("bert")\
 .setCaseSensitive(False)\
 .setDimension(768)\
 .setStorageRef('bert_base_uncased') 

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [12]:
bert.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [13]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your BERT model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀 

In [14]:
! ls -l {MODEL_NAME}_spark_nlp

total 396680
-rw-r--r-- 1 root root 406188124 Jun 20 21:39 bert_tensorflow
drwxr-xr-x 4 root root      4096 Jun 20 21:36 fields
drwxr-xr-x 2 root root      4096 Jun 20 21:36 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny BERT model 😊 

In [15]:
"""bert_loaded = BertEmbeddings.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["sentence",'token'])\
  .setOutputCol("bert")\
  .setCaseSensitive(False)"""

'bert_loaded = BertEmbeddings.load("./{}_spark_nlp".format(MODEL_NAME))  .setInputCols(["sentence",\'token\'])  .setOutputCol("bert")  .setCaseSensitive(False)'

In [16]:
bert.getStorageRef()

'bert_base_uncased'

That's it! You can now go wild and use hundreds of BERT models from HuggingFace 🤗 in Spark NLP 🚀 


In [17]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd

In [18]:
newsDF= pd.read_csv('/content/drive/MyDrive/Madar/Pyspark (nlp & voce) /final_dataset84k.csv')


In [19]:
newsDF.dtypes

Unnamed: 0        int64
Body as text     object
label            object
dtype: object

In [20]:

a = ["1","2","0"]

newsDF = newsDF[newsDF['label'].isin(a)]

In [21]:
newsDF['label'].value_counts()

2    66412
1     9981
0     7568
Name: label, dtype: int64

In [22]:
newsDF = newsDF.astype({"Body as text ": str, "label": int})


In [23]:
newsDF = spark.createDataFrame(newsDF)
newsDF .show()

+----------+--------------------+-----+
|Unnamed: 0|       Body as text |label|
+----------+--------------------+-----+
|         0|J'ai bien lu votr...|    2|
|         1|Comme discuté; le...|    2|
|         2|Faiez Boudabbous ...|    0|
|         3|           Ok, merci|    1|
|         4|Bonsoir Tasnim, c...|    2|
|         5|    Je vous remercie|    1|
|         6|      Avec plaisir !|    1|
|         7|Timchy ma3a firefire|    1|
|         8|Composez *140# et...|    2|
|         9|T3ty les points m...|    1|
|        10|Bonsoir Hamdi, vo...|    2|
|        11|Bonsoir Adel, vou...|    2|
|        12|Bonsoir Achref, q...|    2|
|        13|             Bonsoir|    1|
|        14|Bonsoir Ghassen, ...|    2|
|        15|Bllhy na7ouli off...|    2|
|        16|                سلام|    1|
|        17|Bonsoir Chrif, su...|    2|
|        18|        بربي شنسألكم|    1|
|        19|إنشاء الله مربوحة...|    1|
+----------+--------------------+-----+
only showing top 20 rows



In [24]:


from pyspark.sql.types import IntegerType
#df = df.astype({"Body as text ": str, "label": int})
newsDF = newsDF.withColumnRenamed("Body as text ","text")
newsDF = newsDF.withColumn("label", newsDF["label"].cast(IntegerType()))
(trainDataset, testDataset) = newsDF.randomSplit([0.8, 0.2], seed = 100)

In [25]:
"""document_assembler = DocumentAssembler() \
    .setInputCol("Body as text ") \
    .setOutputCol("document")
    
tokenizer = Tokenizer() \
  .setInputCols(["document"]) \
  .setOutputCol("token")
    

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("token")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)
"""

'document_assembler = DocumentAssembler()     .setInputCol("Body as text ")     .setOutputCol("document")\n    \ntokenizer = Tokenizer()   .setInputCols(["document"])   .setOutputCol("token")\n    \n\nstopwords_cleaner = StopWordsCleaner()      .setInputCols("token")      .setOutputCol("cleanTokens")      .setCaseSensitive(False)\n'

In [26]:
"""
bert_loaded = BertEmbeddings.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["body as text ",'token'])\
  .setOutputCol("embeddings")\
  .setCaseSensitive(False)
  
  """ 

'\nbert_loaded = BertEmbeddings.load("./{}_spark_nlp".format(MODEL_NAME))  .setInputCols(["body as text ",\'token\'])  .setOutputCol("embeddings")  .setCaseSensitive(False)\n  \n  '

In [27]:
import findspark
findspark.add_packages('mysql:mysql-connector-java:8.0.11')

In [28]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

sentenceDetector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

regexTokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

embeddings = BertEmbeddings.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["sentence",'token'])\
  .setOutputCol("bert")\
  .setCaseSensitive(False)

classsifierdl = ClassifierDLApproach()\
  .setInputCols(["bert"])\
  .setOutputCol("label_pred")\
  .setLabelColumn("label")\
  .setMaxEpochs(20)\
  .setLr(0.001)\
  .setBatchSize(16)\
  .setEnableOutputLogs(True)



In [29]:
pipeline = Pipeline().setStages([documentAssembler, sentenceDetector, regexTokenizer, embeddings,classsifierdl])

In [30]:
results = pipeline.fit(trainDataset)

Py4JJavaError: ignored

In [ ]:
!ls -alh /root/annotator_logs

In [ ]:
!cat /root/annotator_logs/ClassifierDLApproach_e154e89ed272.log

In [ ]:

results  =  pipeline.transform(trainDataset)

In [ ]:
!pip install scikit-learn==0.16.1
from sklearn.metrics import classification_report

In [ ]:
preds_df = results.select('label','texts',"label_pred").toPandas()

preds_df['label_pred'] = preds_df['label_pred'].apply(lambda x : x[0])

In [ ]:
print (classification_report(preds_df['label'], preds_df['label_pred']))